In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 15.5 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=db68a1e844ad4919673d1d9d9e93b20c308ffbff046bba0b6a815abebee47e08
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, IntegerType, DateType, DoubleType
from pyspark.sql import functions

In [5]:
spark = SparkSession \
    .builder \
    .appName("airlinedelay") \
    .getOrCreate()

context = spark.sparkContext

In [6]:
schema = StructType([
    StructField("FL_DATE", DateType(), True),
    StructField("OP_CARRIER", StringType(), True),
    StructField("OP_CARRIER_FL_NUM", IntegerType(), True),
    StructField("ORIGIN", StringType(), True),
    StructField("DEST", StringType(), True),
    StructField("CRS_DEP_TIME", DoubleType(), True),
    StructField("DEP_TIME", DoubleType(), True),
    StructField("DEP_DELAY", DoubleType(), True),
    StructField("TAXI_OUT", DoubleType(), True),
    StructField("WHEELS_OFF", DoubleType(), True),
    StructField("WHEELS_ON", DoubleType(), True),
    StructField("TAXI_IN", DoubleType(), True),
    StructField("CRS_ARR_TIME", DoubleType(), True),
    StructField("ARR_TIME", DoubleType(), True),
    StructField("ARR_DELAY", DoubleType(), True),
    StructField("CANCELLED", DoubleType(), True),
    StructField("CANCELLATION_CODE", StringType(), True),
    StructField("DIVERTED", DoubleType(), True),
    StructField("CRS_ELAPSED_TIME", DoubleType(), True),
    StructField("ACTUAL_ELAPSED_TIME", DoubleType(), True),
    StructField("AIR_TIME", DoubleType(), True),
    StructField("DISTANCE", DoubleType(), True),
    StructField("CARRIER_DELAY", DoubleType(), True),
    StructField("WEATHER_DELAY", DoubleType(), True),
    StructField("NAS_DELAY", DoubleType(), True),
    StructField("SECURITY_DELAY", DoubleType(), True),
    StructField("LATE_AIRCRAFT_DELAY", DoubleType(), True)
])

df = spark.read.format("csv") \
    .option("sep", ",") \
    .option("header", True) \
    .schema(schema) \
    .load("./drive/MyDrive/bda_airline_delay_analysis/*.csv")

df.createOrReplaceTempView("airlinedelay")

In [7]:
df.show(1000)

+----------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+-----------------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+-------------------+
|   FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|CANCELLATION_CODE|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|
+----------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+-----------------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+-------------------+
|2009-01-01|        XE|             1

In [8]:
spark.sql("SELECT count(*) FROM airlinedelay").show()

+--------+
|count(1)|
+--------+
|18946105|
+--------+



Departure location with the most delays

In [19]:
spark.sql("SELECT ORIGIN, SUM(IF(airlinedelay.DEP_DELAY > 0, 1, 0)) AS CountDelaysPerOrigin FROM airlinedelay group by ORIGIN order by CountDelaysPerOrigin DESC").show()

+------+--------------------+
|ORIGIN|CountDelaysPerOrigin|
+------+--------------------+
|   ATL|              466039|
|   ORD|              359808|
|   DFW|              332197|
|   DEN|              302637|
|   IAH|              232171|
|   LAX|              227453|
|   PHX|              210542|
|   LAS|              194268|
|   SFO|              164931|
|   EWR|              149817|
|   DTW|              147050|
|   BWI|              146797|
|   MDW|              145507|
|   MCO|              140559|
|   CLT|              133253|
|   MSP|              121196|
|   SLC|              120717|
|   JFK|              113054|
|   BOS|              106483|
|   PHL|               98436|
+------+--------------------+
only showing top 20 rows



Company with the most delays

In [20]:
spark.sql("SELECT OP_CARRIER, SUM(IF(airlinedelay.DEP_DELAY > 0, 1, 0)) AS CountDelaysPerCompany FROM airlinedelay group by OP_CARRIER order by CountDelaysPerCompany DESC").show()

+----------+---------------------+
|OP_CARRIER|CountDelaysPerCompany|
+----------+---------------------+
|        WN|              1812981|
|        DL|               676100|
|        AA|               599841|
|        OO|               535132|
|        MQ|               435585|
|        CO|               355614|
|        US|               344752|
|        XE|               341966|
|        EV|               322065|
|        UA|               313426|
|        FL|               224293|
|        B6|               221426|
|        9E|               137897|
|        YV|               136454|
|        AS|               109208|
|        F9|               103981|
|        NW|                95447|
|        OH|                74176|
|        HA|                42777|
+----------+---------------------+



Locations of departure with most flights

In [21]:
spark.sql("SELECT ORIGIN, COUNT(*) as DepartureFlights FROM airlinedelay group by ORIGIN order by DepartureFlights DESC").show()

+------+----------------+
|ORIGIN|DepartureFlights|
+------+----------------+
|   ATL|         1224072|
|   ORD|          940001|
|   DFW|          791723|
|   DEN|          714556|
|   LAX|          602621|
|   PHX|          552724|
|   IAH|          539599|
|   LAS|          445120|
|   SFO|          421656|
|   DTW|          401928|
|   SLC|          375267|
|   CLT|          370605|
|   MCO|          366902|
|   MSP|          349450|
|   EWR|          344267|
|   BOS|          333349|
|   JFK|          325554|
|   BWI|          310868|
|   LGA|          300730|
|   SEA|          299191|
+------+----------------+
only showing top 20 rows



Locations of arrivals with most flights

In [23]:
spark.sql("SELECT DEST, COUNT(*) as ArrivalFlights FROM airlinedelay group by DEST order by ArrivalFlights DESC").show()

+----+--------------+
|DEST|ArrivalFlights|
+----+--------------+
| ATL|       1224186|
| ORD|        940098|
| DFW|        791735|
| DEN|        714664|
| LAX|        602728|
| PHX|        552668|
| IAH|        539593|
| LAS|        445084|
| SFO|        421706|
| DTW|        401833|
| SLC|        375300|
| CLT|        370624|
| MCO|        366903|
| MSP|        349508|
| EWR|        344285|
| BOS|        333335|
| JFK|        325511|
| BWI|        310850|
| LGA|        300713|
| SEA|        299153|
+----+--------------+
only showing top 20 rows



Companies with most carriers deployed

In [27]:
spark.sql("SELECT OP_CARRIER, COUNT(OP_CARRIER_FL_NUM) as CountOfCarriers FROM airlinedelay group by OP_CARRIER order by CountOfCarriers DESC").show()

+----------+---------------+
|OP_CARRIER|CountOfCarriers|
+----------+---------------+
|        WN|        3407373|
|        DL|        1888486|
|        OO|        1730957|
|        AA|        1627708|
|        MQ|        1319921|
|        US|        1227270|
|        XE|        1105864|
|        UA|        1029251|
|        EV|         945475|
|        FL|         746780|
|        CO|         740498|
|        B6|         610258|
|        YV|         573522|
|        9E|         523633|
|        AS|         417340|
|        OH|         298603|
|        NW|         292400|
|        F9|         253858|
|        HA|         206908|
+----------+---------------+



Companies with best average taxying out time

In [29]:
spark.sql("SELECT OP_CARRIER, AVG(TAXI_OUT) as AvgTaxiOutTime FROM airlinedelay group by OP_CARRIER order by AvgTaxiOutTime ASC").show()

+----------+------------------+
|OP_CARRIER|    AvgTaxiOutTime|
+----------+------------------+
|        HA|  9.94550330891786|
|        WN| 10.34115725504928|
|        F9|14.139743539003444|
|        AS|14.560969608668488|
|        OO|14.722402030368498|
|        FL|  14.8211612892756|
|        XE|  15.3352979085434|
|        MQ|15.500842486319366|
|        AA|15.695579027307058|
|        YV|15.714004629011914|
|        UA| 16.47260687581034|
|        EV|  17.2854259880585|
|        CO|17.774577367457123|
|        9E| 18.52385922733359|
|        US|  18.5897494403164|
|        B6| 19.04905968110707|
|        NW|19.202903536535022|
|        DL| 20.27339690792086|
|        OH|23.089396302303854|
+----------+------------------+



Carriers with best average times(ARR_TIME - DEP_TIME) in minutes

In [34]:
spark.sql("SELECT OP_CARRIER_FL_NUM, AVG(ABS((ABS(ARR_TIME/100)*60 +(ARR_TIME%100)) - (ABS(DEP_TIME/100)*60 +(DEP_TIME%100)))) as AvgTripTime FROM airlinedelay group by OP_CARRIER_FL_NUM order by AvgTripTime ASC").show()

+-----------------+------------------+
|OP_CARRIER_FL_NUM|       AvgTripTime|
+-----------------+------------------+
|             8942|              null|
|             7762|16.814035087719297|
|             4977|19.088015340364333|
|             8991| 26.40000000000009|
|             5253|29.757552581261947|
|             5093|  32.8815766923736|
|             4312| 33.37064471879287|
|             4106| 37.29795918367346|
|             6143| 39.86333333333334|
|             4190| 41.16059701492537|
|             7823| 41.60000000000002|
|             4209|42.147337709700956|
|             4398| 42.72483959670028|
|             4176|42.859317585301845|
|             4152| 44.91201982651797|
|             4160|45.876015936254994|
|             6782| 46.13333333333336|
|             4789| 46.38662613981763|
|             8941|46.599999999999966|
|             6942| 47.41838440111421|
+-----------------+------------------+
only showing top 20 rows

